<a href="https://colab.research.google.com/github/Linomc/olimpiadas_unirio/blob/main/origem_alunos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Origem Alunos

## Síntese: Por iniciativa própria dos alunos que estão inscritos na olimpiada de inovação e estão no mesmo grupo de comunicação, criaram uma enquete (com variações) com: nome, nível e curso.

In [ ]:
! pip install scipy
! pip install gspread
! pip install gspread-dataframe

In [ ]:
# vi que não tinha a biblioteca para ods por isso baixei essas duas.
!pip install pandas-ods-reader
!pip install odfpy

In [ ]:
# biblioteca q acho q vale a pena para saídas visuais
!pip install dash

In [5]:
from pandas import DataFrame, read_csv, read_excel, crosstab, pivot_table
from google.colab import drive, files, auth
import gspread
from scipy import stats
import numpy as np

In [7]:
# vamos pegar a planilha
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [6]:
drive.mount('/gdrive')
auth.authenticate_user()

Mounted at /gdrive


### Baixando a planilha e criando o quadro de dados (data frame)

In [8]:
from oauth2client import client
tabela_enq = gc.open_by_url('https://docs.google.com/spreadsheets/d/1D1ycnmx49cA1hAqhzozYqcLFtvcIMlslhFAZg0Qjcj0/edit?usp=sharing').sheet1
dados = tabela_enq.get_all_values()
df_enquete = DataFrame(dados)
df_enquete.set_axis(df_enquete.loc[0], axis='columns', inplace=True)
df_enquete.drop(0, axis=0, inplace=True)
#df_enquete.reset_index(drop=True)

<ipython-input-8-3f6dc0c71233>:5: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df_enquete.set_axis(df_enquete.loc[0], axis='columns', inplace=True)


In [9]:
df_enquete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 1 to 25
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genero        25 non-null     object
 1   nivel         25 non-null     object
 2   aluno_unirio  25 non-null     object
 3   escola        25 non-null     object
dtypes: object(4)
memory usage: 932.0+ bytes


In [10]:
df_enquete.tail(1)

,genero,nivel,aluno_unirio,escola
25,f,graduacao,museologia,museologia


### Criando tabela de contingência com as variáveis: genero, escola

In [11]:
df_escola_genero = df_enquete[['genero','escola']].groupby(['genero']).value_counts().reset_index().rename(columns = {0:'alun@s'})
df_escola_genero.columns

Index(['genero', 'escola', 'alun@s'], dtype='object')

In [12]:
tabela_contingencia = pivot_table(df_escola_genero, values = 'alun@s',index=['escola'], columns=['genero'], aggfunc='sum', fill_value= 0).reset_index()
tabela_contingencia

genero,escola,f,m
0,administracao publica,2,2
1,biblioteconomia,1,0
2,ciencias biologicas,2,0
3,enfermagem,1,1
4,informatica,0,1
5,matematica,0,2
6,museologia,4,2
7,producao,6,0
8,teatro,0,1


In [13]:
escolas_contagem = DataFrame(df_enquete['escola'].value_counts())
escolas_contagem = escolas_contagem.reset_index().rename(columns={"index": "escolas", "escola": "alun@s"})
escolas_contagem

,escolas,alun@s
0,museologia,6
1,producao,6
2,administracao publica,4
3,matematica,2
4,ciencias biologicas,2
5,enfermagem,2
6,informatica,1
7,biblioteconomia,1
8,teatro,1


In [14]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import plotly.express as px

In [15]:
#saída gráfica
fig = px.bar(tabela_contingencia, x= 'escola', y = ['f','m'], title = 'Amostra da população dos participantes por escola')
fig.show()

In [16]:
(df_enquete['nivel'].value_counts()/len(df_enquete['nivel']))*100

graduacao    76.0
pos          24.0
Name: nivel, dtype: float64

In [17]:
(df_enquete['escola'].value_counts()/len(df_enquete['nivel']))*100

museologia               24.0
producao                 24.0
administracao publica    16.0
matematica                8.0
ciencias biologicas       8.0
enfermagem                8.0
informatica               4.0
biblioteconomia           4.0
teatro                    4.0
Name: escola, dtype: float64

In [ ]:
app = Dash(__name__)

app.layout = html.Div([
    html.Div(children='origem dos participantes'),
    html.Hr(),
    dcc.RadioItems(options=['genero', 'nivel', 'escola'], value= '')
])